In [1]:
import geopandas as gpd
import pandas as pd

from fwi_predict.constants import WQ_RANGES
from fwi_predict.wq import check_in_range

In [2]:
ponds = gpd.read_file("../data/clean/pond_metadata_clean.geojson")
measurements = pd.read_csv("../data/clean/ara_measurements_clean.csv",
                                parse_dates=['sample_dt', 'prescribed_collection_date'])

In [3]:
# Subset to columns of interest
ponds_cols = ['pond_id', 'property_area_acres', 'pond_area_acres',
                      'pond_depth_meters', 'geometry']
pond_measurement_cols = ['pond_id', 'region', 'farm_id', 'group', 'treatment_group',
                         'sample_dt', 'time_of_day', 'do_mg_per_L', 'ph', 'turbidity_cm',
                         'ammonia_mg_per_L', 'fish_per_acre', 'species', 'primary_productivity_gpp_mg_per_L']

# Construct days since last measurement variable. This would suggest including water quality correction

In [4]:
ponds = ponds[ponds_cols]
ponds = ponds[ponds['geometry'].notna()] # Have to have locations to get weather data.
measurements = measurements[measurements['follow_up'] == False]
measurements = measurements[pond_measurement_cols]

In [5]:
# Remove measurements with inconsistent times of day
measurements['tod_dt'] = measurements['sample_dt'].apply(lambda x: "morning" if x.hour < 12 else "evening")
print(f"Observations with inconsistent times: {len(measurements[measurements['tod_dt'] != measurements['time_of_day']])}")
measurements = measurements[measurements['tod_dt'] == measurements['time_of_day']]
measurements = measurements.drop(columns=['tod_dt'])

Observations with inconsistent times: 44


In [6]:
predict_ds = gpd.GeoDataFrame(
  measurements.merge(ponds, on='pond_id', how='inner'), # Inner as we need location
  geometry='geometry',
  crs=ponds.crs
)

In [7]:
ponds_not_included = ponds[~ponds['pond_id'].isin(predict_ds['pond_id'])]
print(f"No measurements for {len(ponds_not_included)} ponds")

No measurements for 64 ponds


In [8]:
for wq_param in WQ_RANGES.keys():
  param_no_unit = wq_param.split('_', 1)[0]
  predict_ds[f"{param_no_unit}_in_req_range"] = check_in_range(wq_param, predict_ds[wq_param], predict_ds['time_of_day'])

C:\Users\squaade\Documents\Github\fwi-predict\fwi_predict\wq.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  in_range[values.isna() | periods.isna()] = np.nan
C:\Users\squaade\Documents\Github\fwi-predict\fwi_predict\wq.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  in_range[values.isna()] = np.nan
C:\Users\squaade\Documents\Github\fwi-predict\fwi_predict\wq.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  in_range[values.isna()] = np.nan
C:\Users\squaa

In [9]:
predict_ds.describe().T

,count,mean,std,min,25%,50%,75%,max
do_mg_per_L,4877.0,6.500465,3.624134,0.00,3.60,4.80,9.40,80.80
ph,4877.0,8.182413,0.314831,3.38,8.00,8.21,8.39,9.54
turbidity_cm,3414.0,28.950644,7.524521,9.00,24.00,29.00,32.00,89.00
ammonia_mg_per_L,2965.0,0.131960,0.257797,0.00,0.01,0.06,0.18,9.00
fish_per_acre,2868.0,5118.866806,8632.648089,0.00,2300.00,3000.00,3882.00,66667.00
primary_productivity_gpp_mg_per_L,1202.0,3.440512,2.700201,0.00,1.85,3.00,4.70,16.90
property_area_acres,4969.0,9.259086,10.982728,0.25,2.50,5.00,13.00,53.00
pond_area_acres,4194.0,8.298865,10.499382,0.13,1.65,3.45,10.81,49.80
pond_depth_meters,4969.0,2.163614,0.808669,1.00,1.90,2.00,2.50,11.00


In [10]:
predict_ds['measurement_idx'] = pd.Series([i for i in range(0, len(predict_ds))])
predict_ds.to_file("../data/clean/ara_predict_ds.geojson")

In [11]:
predict_ds.columns

Index(['pond_id', 'region', 'farm_id', 'group', 'treatment_group', 'sample_dt',
       'time_of_day', 'do_mg_per_L', 'ph', 'turbidity_cm', 'ammonia_mg_per_L',
       'fish_per_acre', 'species', 'primary_productivity_gpp_mg_per_L',
       'property_area_acres', 'pond_area_acres', 'pond_depth_meters',
       'geometry', 'do_in_req_range', 'ph_in_req_range',
       'ammonia_in_req_range', 'turbidity_in_req_range', 'measurement_idx'],
      dtype='object')